In [1]:
import pandas as pd
import numpy as np
import os
from IPython.display import display, Audio
import librosa
import random
import whisper
import adapters

/work/dpandya/miniconda3/envs/onlyWhisper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [103]:
from transformers import AutomaticSpeechRecognitionPipeline
from transformers import WhisperFeatureExtractor, WhisperTokenizer, pipeline, WhisperProcessor
import torch

model_name = 'openai/whisper-large-v3'

feature_extractor = WhisperFeatureExtractor.from_pretrained(model_name, language="english", task="transcribe", sample_rate=16000)
tokenizer = WhisperTokenizer.from_pretrained(model_name, language="english", task="transcribe")
processor = WhisperProcessor.from_pretrained(model_name, language="english", task="transcribe")

#model = adapters.WhisperAdapterModel.from_pretrained(model_name, language="english", task="transcribe")
model = adapters.WhisperAdapterModel.from_pretrained(model_name)
#model.config.max_length = 512
#model.config.use_cache = False
'''adapter_name = model.load_adapter(
    '/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/seqBN_r16/',
    set_active=True
)
#model.config.max_length=512
model.set_active_adapters(adapter_name)'''
#ddf = pd.read_csv('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/eval_seqBN_r8.csv')

"adapter_name = model.load_adapter(\n    '/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/seqBN_r16/',\n    set_active=True\n)\n#model.config.max_length=512\nmodel.set_active_adapters(adapter_name)"

In [107]:
model.generation_config._from_model_config

False

In [ ]:
'''model.config.forced_decoder_ids = None
model.config.suppress_tokens = []
model.generation_config.forced_decoder_ids = None
model.generation_config._from_model_config = True'''

In [89]:
model.generation_config.input_ids = model.generation_config.forced_decoder_ids
model.generation_config.forced_decoder_ids = None
model.generation_config.alignment_heads = [[2, 2], [3, 0], [3, 2], [3, 3], [3, 4], [3, 5]]

In [104]:
import torch

eval_pipeline = AutomaticSpeechRecognitionPipeline(
            model=model,
            feature_extractor=feature_extractor,
            tokenizer=tokenizer,
            device="cuda:0", # Use GPU 0, or -1 for CPU
)
#forced_decoder_ids = processor.get_decoder_prompt_ids(language="english", task="transcribe")

Device set to use cuda:0


In [97]:
def add_audio_dir(s):
    return os.path.join('/ceph/dpandya/notsofar/nsfd_adap_segments/eval/', s)

df = pd.read_csv("/ceph/dpandya/notsofar/nsfd_adap_segments/eval_segments.csv")
df['segmented_audio_file'] = df['segmented_audio_file'].apply(add_audio_dir)

In [105]:
eval_pipeline(list(df['segmented_audio_file'])[0:3])

ValueError: You have explicitly specified `forced_decoder_ids`. Please remove the `forced_decoder_ids` argument in favour of `input_ids` or `decoder_input_ids` respectively.

In [8]:
df = pd.read_csv("/ceph/dpandya/notsofar/nsfd_adap_segments/eval_segments.csv")
df['duration'] = (df['stop'] - df['start'])/1000

In [75]:
from transformers import WhisperForConditionalGeneration

type(WhisperForConditionalGeneration.from_pretrained('openai/whisper-small').model)

transformers.models.whisper.modeling_whisper.WhisperModel

In [9]:
import random

df.sample(540, random_state=42)

,original_audio_file,segmented_audio_file,segmented_text,start,stop,duration
881,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32049_ch0_segment_16.wav,And <ST/> And I'll check when that uh scientis...,356500.0,356890.0,0.39
453,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32331_ch0_segment_10.wav,"Uh, the trees <ST/> <ST/> still have some leav...",210130.0,232440.0,22.31
1844,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32180_ch0_segment_3.wav,I have an idea about something that might be v...,48610.0,68750.0,20.14
1353,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32087_ch0_segment_15.wav,<ST/> and uh <FILL/> just to get as many diffe...,351810.0,352950.0,1.14
281,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32000_ch0_segment_13.wav,<ST/> this <ST/> <UNKNOWN/> <ST/> meeting? Wha...,283220.0,307100.0,23.88
...,...,...,...,...,...,...
244,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32054_ch0_segment_12.wav,"<ST/> and that, we rent one site per day or is...",256070.0,280160.0,24.09
997,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32086_ch0_segment_1.wav,Hi <PName> Rachel </PName> . Welcome. <BA/> Th...,6740.0,31450.0,24.71
1422,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32312_ch0_segment_7.wav,Refrigerator magnets. I think <ST/> <ST/> on t...,145830.0,168830.0,23.00
1691,/ceph/dpandya/notsofar/eval_set/240825.1_eval_...,MTG_32322_ch0_segment_6.wav,Um. Yeah I would like to actually <ST/> <ST/> ...,113010.0,137610.0,24.60


In [2]:
model = adapters.WhisperAdapterModel.from_pretrained(
    "openai/whisper-medium",
    language="english",
    task="transcribe",
)

2025-11-04 13:56:46.124199: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-04 13:56:46.124262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-04 13:56:46.125895: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-04 13:56:46.135069: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-04 13:56:46.884748: W tensorflow/compiler/tf2

In [4]:
model.load_adapter(
    "/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/seqBN_r16/"
)

'seqBN_r16'

In [6]:
d_dir = "/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/"
dd = os.listdir(d_dir)
for d in dd:
    if os.path.isdir(os.path.join(d_dir, d)):
        print(d)

seqBN_r8
seqBN_r32
seqBN_r16
seqBN_r4


In [16]:
results = pd.read_json('/ceph/dpandya/notsofar/nsfd_adap_segments/adapter_base/medium/experiment_results.json')
rf4 = pd.DataFrame(results['seqBN_r32']['log_history'])[:-1]

- /ceph/dpandya/notsofar/nsfd_all.csv -- all the audio files
- 

In [14]:
#pd.read_csv('/ceph/dpandya/notsofar/dev_set/240825.1_dev1/segmented_audios.csv')
pd.read_csv('/ceph/dpandya/notsofar/newNotsofar/train.csv')

,audio_files,transcription_file,mic
0,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_rockfall_1
1,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_plaza_0
2,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_rockfall_1
3,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_rockfall_2
4,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_rockfall_0
...,...,...,...
67,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_rockfall_2
68,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_plaza_0
69,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_meetup_0
70,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,sc_plaza_0


In [8]:
import re

def clean_text(s):
    # Convert to string to handle potential non-string inputs gracefully
    s = str(s)
    
    # 1. Remove text enclosed in angle brackets (e.g., <ST>, <UNKNOWN>)
    s = re.sub(r'<[^>]+>', '', s)
    
    # 2. Remove punctuation
    # This regex matches any character that is NOT a word character (alphanumeric + underscore) or whitespace
    s = re.sub(r'[^\w\s]', '', s)
    
    # 3. Remove extra spaces (replace multiple spaces with a single space)
    s = re.sub(r'\s+', ' ', s)
    
    # 4. Remove leading and trailing spaces
    s = s.strip()
    
    # 5. Convert to lowercase for case-insensitive comparison
    return s.lower()

In [9]:
samp = '/ceph/dpandya/notsofar/eval_set/240825.1_eval_full_with_GT/MTG/'
df = pd.read_csv(os.path.join(samp, 'eval.csv'))
trSample = pd.read_json(df['gt_transcription_files'].iloc[0])
trSample['text'].iloc[1], clean_text(trSample['text'].iloc[1])

('<ST/> we got a <ST/> we got a big budget in the office uh regarding snacks uh yeah <ST/>',
 'we got a we got a big budget in the office uh regarding snacks uh yeah')

In [80]:
#d = pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/segmented_audios.csv')
#pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train')
#pd.read_csv('/ceph/dpandya/notsofar/nsfd_all.csv')
print(os.listdir('/ceph/dpandya/notsofar/train_set/small_train_features/medium/'))
#pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/segmented_audios.csv')

['dataset_info.json', 'data-00000-of-00001.arrow', 'cache-941e4f085085b177.arrow', 'state.json']


In [11]:
dd = pd.read_csv('/ceph/dpandya/notsofar/train_set/240825.1_train/new_segmented_audios/new_segmented_audios.csv')
dd.dropna(subset=['text'])

,org_audio_file,segmented_audio_file,text
0,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,alright everyone let us continue on with our ...
1,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,have any of you guys used it yet? um not not y...
2,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,i would i would love to try it to see. but is ...
3,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,so you don't have to have someone monitoring y...
4,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,"oh i see, oh i see, ok. and you could just to ..."
...,...,...,...
6776,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,i think both is is a good option. you think bo...
6777,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,there could be um like toys and things for kid...
6778,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,what i feel wasn't set up an ice cream cone li...
6779,/ceph/dpandya/notsofar/train_set/240825.1_trai...,/ceph/dpandya/notsofar/train_set/240825.1_trai...,maybe like each level like the first level is ...


- randomly select a single-channel audio stream from the meetings
- compile a list of these meetings
- create a df with their transcription files
- Divide them into train and eval sets

In [10]:
train_dir =  '/ceph/dpandya/notsofar/train_set/240825.1_train/MTG/'
eval_dir = '/ceph/dpandya/notsofar/eval_set/240825.1_eval_full_with_GT/MTG/'
train_meetings = os.listdir('/ceph/dpandya/notsofar/train_set/240825.1_train/MTG/')
eval_meetings = os.listdir('/ceph/dpandya/notsofar/eval_set/240825.1_eval_full_with_GT/MTG/')

In [20]:
train_files = []
transcription_files = []
mic = []
# Iterate through all the train meeting dir and choosing a random single channel audio stream
for meet in train_meetings:
    stream_list = []
    # Collecting all the streams for a meeting
    for stream in os.listdir(os.path.join(train_dir, meet)):
        if os.path.isdir(os.path.join(train_dir, meet, stream)):
            if stream.split('_')[0] == 'sc':
                stream_list.append(os.path.join(train_dir, meet, stream, 'ch0.wav'))
    random_ = random.choice(stream_list)
    
    mic.append(random_.split('/')[-2])
    train_files.append(random_)
    transcription_files.append(os.path.join(train_dir, meet, 'gt_transcription.json'))
    

In [31]:
# Do the same process for the eval files, choosing a random single channel audio stream
eval_files = []
eval_trans = []
eval_mic = []
# Iterate through all the eval meeting dir and choosing a random single channel audio stream
for meet in eval_meetings:
    stream_list = []
    # Collecting all the streams for a meeting
    if os.path.isdir(os.path.join(eval_dir, meet)):
        for stream in os.listdir(os.path.join(eval_dir, meet)):
            if os.path.isdir(os.path.join(eval_dir, meet, stream)):
                if stream.split('_')[0] == 'sc':
                    stream_list.append(os.path.join(eval_dir, meet, stream, 'ch0.wav'))
        random_ = random.choice(stream_list)
    
        eval_mic.append(random_.split('/')[-2])
        eval_files.append(random_)
        eval_trans.append(os.path.join(eval_dir, meet, 'gt_transcription.json'))

In [35]:
train_df = pd.DataFrame()
train_df['audio_files'] = train_files
train_df['transcription_file'] = transcription_files
train_df['mic'] = mic

test_df = pd.DataFrame()
test_df['audio_files'] = eval_files
test_df['transcription_files'] = eval_trans
test_df['mic'] = eval_mic

In [46]:
train_df.to_csv('/ceph/dpandya/notsofar/newNotsofar/train.csv', index=False)
test_df.to_csv('/ceph/dpandya/notsofar/newNotsofar/test.csv', index=False)